In [ ]:
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")

efficiency_csp_power_block = 0.412  # from data/efficiencies.csv

nice_names = {
    "Curtailed energy output [MWh] from solar-utility": "PV curtailed",
    "Curtailed energy output [MWh] from offwind": "Wind offshore curtailed",
    "Curtailed energy output [MWh] from onwind": "Wind ofshore curtailed",
    "Curtailed energy output [MWh] from csp-tower": "CSP curtailed",
    "Total energy output [MWh] from csp-tower": "CSP",
    "Total energy output [MWh] from offwind": "Wind offshore",
    "Total energy output [MWh] from onwind": "Wind onshore",
    "Total energy output [MWh] from solar-utility": "PV",
}

nice_colors = {
    "CSP": "orange",
    "PV": "#f9d002",
    "Wind onshore": "#235ebc",
    "Wind offshore": "#6895dd",
    "Curtailed": "grey",
}

# Select data to plot
df = pd.read_csv(snakemake.input["results"], delimiter=";")
df = df.loc[
    (df["exporter"] == snakemake.wildcards["exporter"])
    & (df["esc"] == snakemake.wildcards["esc"])
]

df["flexibility"] = df["scenario"].str.replace("_.*", "", regex=True)
df["csp"] = df["scenario"].str.contains("with_csp")

# Add sorted category column to df indicating type of flexibility
# then sort by that category to ensure correct order in x-axis of plot
# Note: Reproduces the order given in "flexibilities".

flexibilities = {
    "unbuffered": "Baseload",
    "annually": "Annual",
}

df["flexibility"] = df.flexibility.replace(flexibilities)

df["flexibility"] = pd.Categorical(
    df.flexibility, categories=flexibilities.values(), ordered=True
)

df = df.sort_values("flexibility")

# Restrict to only needed information from now on
df = df.query("category == 'RES'")
df = df.drop(
    columns=["year", "wacc", "esc", "exporter", "importer", "category", "scenario"]
)

# Only select relevant data
electricity = df.loc[df["subcategory"].isin(nice_names.keys())]
electricity = electricity.pivot_table(
    index=["csp", "flexibility"], columns="subcategory", values="value"
)

# Rename to nicer names
electricity = electricity.rename(columns=nice_names)

# data in TWh
electricity /= 1e6

# data in TWh_heat, convert to TWh_e electricity equivalents
electricity["CSP curtailed"] *= efficiency_csp_power_block
electricity["CSP"] *= efficiency_csp_power_block

# Aggregate curtailment from all techs into single entry
electricity = electricity.groupby(lambda x: "Curtailed" if "curtailed" in x else x, axis="columns").sum()

# Reorder for desired legend order
electricity = electricity[["Wind offshore", "Wind onshore", "PV", "CSP", "Curtailed"]]

# Correct sort order for correct plot order
electricity = electricity.sort_index(ascending=[True, False])

# Plotting
fig, ax = plt.subplots(figsize=(6, 3))

electricity.plot.bar(
    stacked=True, ax=ax, width=0.3, color=[nice_colors[t] for t in electricity.columns]
)

# Seperator between ESCs
ax.plot([1.5] * 2, [0, 250], ls="dashed", color="grey")

ax.text(0.5, 110, "No CSP", va="center", ha="center")
ax.text(2.5, 110, "With CSP", va="center", ha="center")

# ax.set_xlabel("Demand matching")
ax.set_xlabel("")
ax.set_ylabel("Electricity [TWh]")

ax.set_xticklabels(["Annual", "Baseload", "Annual", "Baseload"], rotation=0, fontsize="small")
ax.set_yticks(np.arange(0, 200, 20))
# ax.set_yticks(np.arange(25,275,50), minor=True)
# ax.yaxis.grid(which="minor", ls="--")

# Disable verticle grid lines
ax.xaxis.grid(False)

ax.set_ylim(0, 130)  # Magic number 200: Highest value from TN ESC H2 pipeline

# custom legend
legend = ax.legend(
    handles=ax.get_legend_handles_labels()[0][::-1],  # reverse order to match stacking
    labels=ax.get_legend_handles_labels()[1][::-1],  # reverse order to match stacking
    title=None,
    fontsize="small",
    bbox_to_anchor=(1.0, 0),
    loc="lower left",
)

# hatch patterns for first 2 xtick bars (no CSP) and keep remaining bars unhatched
# then multiply by number of different columns plotted to spread across all columns
patterns = ["///"] * 2 + [None] * 2
patterns = patterns*electricity.columns.size

# Overwrite hatches
bars = ax.patches
for bar, hatch in zip(bars, patterns):
    bar.set_hatch(hatch)

fig.tight_layout()

# Saving figure
fig.savefig(snakemake.output["pdf"], dpi=300, bbox_inches="tight")
fig.savefig(snakemake.output["png"], dpi=300, bbox_inches="tight")